## Instalaciones

In [9]:
!pip install ortools

## Librerías

In [10]:
import numpy as np
import pandas as pd
from ortools.sat.python import cp_model

# Variables

## Vector con los tiempos de cada composición

In [11]:
#Vector de tiempos de cada composición
Tiempos = [2, 4, 1, 3, 3, 2, 5, 7, 6]
print("Tamaño: ", len(Tiempos))


Tamaño:  9


In [12]:
print("Tiempos de cada composición: ")
for i in range(len(Tiempos)):
  print("Tiempo de la composición ", i + 1, ": ", Tiempos[i])

Tiempos de cada composición: 
Tiempo de la composición  1 :  2
Tiempo de la composición  2 :  4
Tiempo de la composición  3 :  1
Tiempo de la composición  4 :  3
Tiempo de la composición  5 :  3
Tiempo de la composición  6 :  2
Tiempo de la composición  7 :  5
Tiempo de la composición  8 :  7
Tiempo de la composición  9 :  6


## Matriz booleana con las composiciones y sus participantes:
- Dimensiones: 5x9
  - 5 filas = 1 fila por cada jugador
  - 9 columnas = 1 columna por cada composición

- Valores:
  - 0 = el jugador no participa en la composición j
  - 1 = el jugador participa en la composición j

Link del archivo: https://docs.google.com/spreadsheets/d/1J2g3x6NzTFycFGnih1oQdELV8xV3d5UgluiMrv_4rjQ/edit?usp=sharing

In [13]:
#Leer matriz del archivo
composiciones = pd.read_csv('Matriz booleana composiciones - Hoja 1.csv')

In [14]:
#Mostrar matriz del archivo
composiciones.index += 1
composiciones

,Comp 1,Comp 2,Comp 3,Comp 4,Comp 5,Comp 6,Comp 7,Comp 8,Comp 9
1,1,0,1,1,0,1,1,0,1
2,1,0,1,1,1,1,0,1,0
3,1,1,0,0,1,0,0,1,0
4,1,0,0,0,1,0,1,0,1
5,0,1,0,1,1,1,1,0,0


In [15]:
composiciones.index

RangeIndex(start=1, stop=6, step=1)

In [16]:
#obetenemos la relacion de que composiciones toca cada player.
#Le agregamos a cada player el contador de el tiempo de espera.
players_time = []
for i in range(5):
    players_time.append((composiciones.iloc[[i]].values,0))

In [17]:
print(players_time[0])

(array([[1, 0, 1, 1, 0, 1, 1, 0, 1]]), 0)


# Construcción del Modelo

In [18]:
#Crear CSP
model = cp_model.CpModel()

In [19]:
# #Restricciones
# Orden = []
# Orden = model.NewIntVar(1,9, 'Orden')
# model.NewIntervalVar()
# Tiempo_Total = 0

# #Sumar todos los tiempos de espera
# for i in range(len(Orden)):
#     x = Orden[i] #Tener la composición
#     y = Tiempos[x - 1] #Tener el tiempo de la composición
#     Tiempo_Total += y #Sumar al tiempo total



In [20]:
#Variables
Comps = []

#Creamos una variable por cada composition
for i in range(9):
    c = []
    for x in range(9):
        c += [model.NewBoolVar('C'+str(i+1)+'_'+str(x+1))]
    Comps.append(c)
        
#     Comps.append(model.NewIntVar(1,9,'C'+str(i+1)))

print(Comps[0])

[C1_1(0..1), C1_2(0..1), C1_3(0..1), C1_4(0..1), C1_5(0..1), C1_6(0..1), C1_7(0..1), C1_8(0..1), C1_9(0..1)]


In [21]:
#el segundo [i] es el que indica la posisicon
Comps[7][3]

C8_4(0..1)

In [22]:
players = [0,0,0,0,0]

In [23]:
Comps[8]


[C9_1(0..1),
 C9_2(0..1),
 C9_3(0..1),
 C9_4(0..1),
 C9_5(0..1),
 C9_6(0..1),
 C9_7(0..1),
 C9_8(0..1),
 C9_9(0..1)]

In [24]:
#Restricciones

#Se asegura de que cada composicion solo se toque 1 vez
for i in range(9):
    model.Add(sum(Comps[i]) == 1)

#Cada comp no puede ser al mismo tiempo    
for x in range(9):
    suma = 0
    for y in range(9):
        suma += Comps[y][x]
    model.Add(suma == 1)
#         model.Add(Comps[i][x])
    
#Composition 2 debe ser antes de composition 8    
for i in range(9):
    boo = Comps[7][i]  #encontramos la posisicon del 8vo comp
    model.Add(sum(Comps[1][:i]) == 1).OnlyEnforceIf(boo) #si la posicion de 8vo comp es true entonces todo lo anterior de la comp 2 debe ser falso

#Compisition 6 debe ser inmediatamente despues de comp 5
for i in range(8):
    boo = Comps[4][i] #encontramos la posisicon de la 5ta comp
    model.Add(Comps[5][i+1] == 1).OnlyEnforceIf(boo) 
        
    
# model.AddAllDifferent(Comps)

#Añadiendo el orden a ser respetado
# model.Add(Comps[5] == Comps[4] + 1) #La composicion 6 debe ser tocada inmediatamente despues de la 5
# model.Add(Comps[1] < Comps[7])      #La composicion 2 debe ser tocada antes de la composicion 8





# Solver
Resolver el modelo

In [25]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    for i in range(9):
        print('C'+str(i+1),end=' ')
        for x in range(9):
            print(solver.Value(Comps[i][x]), end='')
        print('')

C1 000000100
C2 010000000
C3 000001000
C4 000100000
C5 000000010
C6 000000001
C7 000010000
C8 001000000
C9 100000000
